In [45]:
import numpy as np
import random
import pandas as pd

In [46]:
students = pd.read_csv('dataset_full.csv')

# hyperparameters
num_individuals = 50
groupsize = 5

Darwinian Natural Selection

- Variation
- Selection
- Heredity

1. Variation (Create a population)

In [47]:
students.head()

,ID,Name,Gender,Preferred language,Majors,Level of ambition,Preferred meeting place,Personality type,Best friend,Preferred day
0,1,Paul Schwartz,Male,German,"('AI', 'CL')",High,Online,ISFP,Mandy Baer,Tuesday
1,2,Mike Maier,Male,Any,"('PHIL', 'AI')",Very high,In person,ESFP,Jürgen Baecker,Thursday
2,3,Phillipp Frei,Male,Any,"('CL', 'NI')",Medium,In person,ENFJ,Jennifer Schiffer,Monday
3,4,Annett Zimmermann,Female,German,"('AI', 'CL')",Low,In person,ESFP,Tobias Brandt,Wednesday
4,5,Manuela Fruehauf,Female,English,"('CL', 'PHIL')",Low,In person,ENFJ,Wolfgang Gärtner,Tuesday


In [48]:
class Population():
    def __init__(self, students, n_individuals, groupsize):
        super(Population, self).__init__()

        if n_individuals % groupsize != 0:
            raise ValueError(f'Received unexpected group size: {n_individuals} groups % {groupsize} individuals has to be 0.')
        
        self.students = students
        self.n_individuals = n_individuals
        self.ids = students.ID.tolist()
        self.groupsize = groupsize

    def create_random_individual(self):
        individual = self.ids.copy()
        random.shuffle(individual)

        return individual

    def create_initial_population(self):
        population = []
        for _ in range(self.n_individuals):
            population.append(self.create_random_individual())

        return np.array(population, dtype=int)

    def show_groups_from_individual(self, individual):

        groups = []
        for i in individual:
            groups.append(self.students['Name'][i-1])
        
        for n,j in enumerate(range(0, len(groups), self.groupsize)):
            print(f'Group {n+1} :',groups[j:j+5])

        return None

In [49]:
population = Population(students, num_individuals, groupsize)

test_population = population.create_initial_population()

test_population

array([[ 54, 100,  67, ...,  42,  72,  78],
       [ 77,  71,   1, ...,  36,  27,  66],
       [ 98,  43,  81, ...,  54,  33,   7],
       ...,
       [ 30,  29,  50, ...,  98,  19,  25],
       [ 56, 100,  65, ...,  87,  28,  64],
       [ 17,  85,  32, ...,  28,  70,  19]])

Show Groups from the IDs

In [50]:
population.show_groups_from_individual(test_population[0])

Group 1 : ['Tobias Brandt', 'Dirk Abend', 'Eric Grunewald', 'Mike Schwab', 'Brigitte Dreher']
Group 2 : ['Eric Neumann', 'Sandra Beike', 'Alexander Gersten', 'Ute Eichelberger', 'Martina Lehmann']
Group 3 : ['Diana Zimmerman', 'Monika Wurfel', 'Bernd Kappel', 'Eric Strauss', 'Michelle Eberhardt']
Group 4 : ['Mandy Baer', 'Sophie Hueber', 'Maik Schulz', 'Marie Drescher', 'Ulrich Lemann']
Group 5 : ['Simone Eichel', 'Patrick Nussbaum', 'Monika Faber', 'Vanessa Waechter', 'Uta Eichel']
Group 6 : ['Jan Gloeckner', 'Yvonne Mahler', 'Jana Kuhn', 'Daniel Neudorf', 'Uta Beyer']
Group 7 : ['Sophie Klug', 'Brigitte Freytag', 'Steffen Konig', 'Klaus Loewe', 'Monika Werner']
Group 8 : ['Anne Maier', 'Annett Zimmermann', 'Leon Schulze', 'Jörg Hertzog', 'Ralph Freytag']
Group 9 : ['Daniel Mueller', 'Klaus Fisher', 'Ines Ritter', 'Max Hartmann', 'Patrick Metzger']
Group 10 : ['Paul Schwartz', 'Antje Koehler', 'Torsten Frei', 'Dirk Zweig', 'Phillipp Frei']
Group 11 : ['Jana Schultheiss', 'Jürgen Koert

________

2. Selection (Evaluate the fitness of each group, find fittest)

In [51]:
class Fitness():
    def __init__(self):
        super(Fitness, self).__init__()

    def evaluate_fitness(self, individual, students):
        """ Evaluates the fitness of an individual. """

        # split individual into student groups of the groupsize
        groups = np.array_split(individual, (len(individual)/groupsize))

        # iterate over groups and calculate scores for the different parameters
        scores = []
        for group_ids in groups:
            # get full data for students in this group from pd dataframe
            group = students.loc[students['ID'].isin(group_ids)]

            # get individual scores for parameters
            language_score = self.evaluate_language(group)
            if language_score == -1:
                # if we really want to use a hard constraint we would need to return -1 here, this makes for really bad (initial) results though
                # return -1
                scores.append(language_score)
                continue

            major_score = self.evaluate_majors(group)
            ambition_score = self.evaluate_ambition(group)
            place_score = self.evaluate_meeting_place(group)
            gender_score = self.evaluate_gender(group)
            friend_score = self.evaluate_friends(group)
            personality_score = self.evaluate_personality(group)
            day_score = self.evaluate_meeting_day(group)

            # formula for adding and weighting different scores
            scores.append(language_score+major_score+ambition_score+place_score+gender_score+friend_score+personality_score+day_score)

        #Convert to series to calculate mean more easily
        return pd.Series(scores).mean()

    def evaluate_language(self, group):
        # number of groupmembers per language
        counts = group['Preferred language'].value_counts()

        # hard constraints, if languages are conflicting, return -1
        if 'German' in counts.index and 'English' in counts.index:
            return -1

        return groupsize

    def evaluate_majors(self, group):
        majors = group['Majors'].tolist()

        # preprocess majors from dataset notation to list with all majors
        group_majors = []
        for pair in majors:
            pair = pair[1:-1].split(", ")

            group_majors.append(pair[0][1:-1])
            group_majors.append(pair[1][1:-1])

        #convert to Series for easier handling
        group_majors = pd.Series(group_majors)
        #get value counts
        group_major_values = group_majors.value_counts()
        #remove majors only one person takes (as they provide no synergy to the group)
        group_major_values = group_major_values[group_major_values > 1]

        #add number of shared majors and divide by 2; Formula is kinda arbitrary
        return group_major_values.sum() /2

    def evaluate_ambition(self, group):
        # get pd Series of ambitions
        ambitions = group['Level of ambition']
        # get int value mappings for ambitions

        mapping = {
            'Very low': 1,
            'Low': 2,
            'Medium': 3,
            'High': 4,
            'Very high': 5
        } 

        ambitions = ambitions.map(mapping)

        # fitness is groupsize - variance in group motivation (so less variance = more fitness)
        return groupsize - ambitions.var()

    def evaluate_meeting_place(self, group):
        # number of groupmembers for each preferred meeting place
        meeting_place = group['Preferred meeting place'].value_counts()

        # if all prefer the same meeting place return 5, else 0
        if meeting_place[0] == groupsize:
            return 5

        return 0

    def evaluate_gender(self, group):
        # evaluate by variance
        genders = group['Gender'].value_counts()

        # add 0 entry for missing genders
        for gender in ['Male', 'Female', 'Indeterminate']:
            if gender not in genders.index:
                genders[gender] = 0

        # return groupsize - variance
        return groupsize - genders.var()

    def evaluate_friends(self, group):
        #for each member +1 if friend is also in group
        group_member_names = group['Name'].tolist()
        best_friends_name = group['Best friend'].tolist()

        # get intersection between both lists
        friends_in_group = list(set(group_member_names).intersection(best_friends_name))

        # fitness += 1 for every pair of friends
        return len(friends_in_group)

    def evaluate_personality(self, group):
        #information about compatible personality types is taken from
        # Montequín, Vicente Rodríguez, et al. "Using Myers-Briggs type indicator (MBTI) as a tool for setting up student teams for information technology projects." Journal of Information Technology and Application in Education 1.1 (2012): 28-34.

        #count existing personality types in each group
        personalities = group['Personality type']
        types = personalities.value_counts()

        #fitness function starts with 0 and gets better
        # with every good group member
        fitness = 0

        #its good if there is a group leader like an ISTJ or an ESTJ, but only one
        try:
            if (types['ISTJ'] + types['ESTJ'] == 1):
                fitness+=5
            elif (types['ISTJ'] + types['ESTJ'] >= 2):
                fitness-=5
        except KeyError:
            pass

        #compare compatibility of group members
        for i, personality_a in enumerate(personalities.tolist()):
            for j, personality_b in enumerate(personalities.tolist()):
                # skip same group member and members already compared
                if i <= j:
                    continue

                # increase fitness if
                if (personality_a[1] != personality_b[1]) ^ (personality_a[2] != personality_b[2]):
                    if (personality_a[0] != personality_b[0]) or (personality_a[3] != personality_b[3]):
                        fitness+=1

        return fitness

    def evaluate_meeting_day(self, group):
        # number of groupmembers for each preferred meeting day
        meeting_day = group['Preferred day'].value_counts()

        # if all prefer the same meeting day return 5, else 0
        if meeting_day[0] == groupsize:
            return 5

        return 0

    def mean_fitness(self, population):
        # get list of fitness scores for all individuals in this population
        fitness_scores = [self.evaluate_fitness(individual, students) for individual in population]

        # convert to series to calculate mean more easily
        return round(pd.Series(fitness_scores).mean(), 3)

    def best_fitness(self, population):
        # get list of fitness scores for all individuals in this population
        fitness_scores = [self.evaluate_fitness(individual, students) for individual in population]

        # sort by best first
        scores_sorted = sorted(fitness_scores, reverse=True)

        # return fitness of best individual
        return round(scores_sorted[0], 3)

    def indices_sorted_by_fitness(self, population):
        fitness_scores = [self.evaluate_fitness(individual, students) for individual in population]

        indices = np.argsort(fitness_scores)

        return indices.tolist()

In [52]:
best_fitness = Fitness().best_fitness(test_population)
best_fitness

19.842

3. Heredity (Pass on the fittest to the next generation)

In [53]:
class Crossover():
    def __init__(self):
        super(Crossover, self).__init__()

    def uniform_order_crossover(self, p1, p2):

        template = self.get_crossover_template(len(p1), crossover_rate=0.2)
        # create 'empty' child
        child = np.zeros((len(p1),),dtype=int)
        # where the template is true, take values from p1
        child[template] = p1[template]
        # store genes used from p1
        used_genes = p1[template]

        # get all genes from p2
        remaining_genes = p2.tolist()
        # add genes from p2 (that were not used from p1) to the empty spots of the child
        for i, value in enumerate(child):
            # if this spot is already filled, continue
            if value != 0:
                continue

            # do while:  pop(get and remove) next gene from p2 until one is found that is not yet in the genome of the child, then add that
            while True:
                next_gene = remaining_genes.pop(0)
                if next_gene not in used_genes:
                    child[i] = next_gene
                    break

        return child

    def get_crossover_template(self,length, crossover_rate = 0.2):
        # initialize template with false values
        template = np.zeros((length,),dtype=bool)
        # get random indices of the amount #of genes * crossover rate
        random_indices = np.random.choice(template.shape[0], int(length*crossover_rate), replace=False)
        #set these indices to true
        template[random_indices] = True

        return template

    def tournament_selection(self, popuplation, tournament_size = 8):
        # get random indices to select random individuals from population
        random_indices = np.random.choice(popuplation.shape[0], tournament_size*2, replace=False)

        # get individuals from random indices and split into two tournaments
        tournament1 = popuplation[random_indices[:tournament_size]]
        tournament2 = popuplation[random_indices[tournament_size:]]

        parents = []
        # tournament is won by fittest individual in each tournament, those become the two parents
        for tournament in (tournament1,tournament2):
            # get fitness scores for every individual in the tournament
            fitness_scores = [Fitness().evaluate_fitness(individual, students) for individual in tournament]
            # get indices ordered by highest fitness first
            idx = np.argsort(fitness_scores)[::-1]
            # add individual with highest fitness to parents
            parents.append(tournament[idx[0]])

        return parents

In [54]:
random_population = Population(students, num_individuals, groupsize).create_initial_population()

print(random_population, '\n')

random_parent1, random_parent2 = Crossover().tournament_selection(random_population)

print(random_parent1, '\n')
print(random_parent2, '\n')

child = Crossover().uniform_order_crossover(random_parent1, random_parent2)

print(child)

[[18 39  3 ... 67 52 68]
 [ 2 42 57 ...  8 99 35]
 [82 99 38 ... 65 73 58]
 ...
 [71 65 26 ... 63 56 75]
 [32 61 19 ...  4 70 12]
 [93 41 14 ... 67 86 58]] 

[ 20  23  76  50  44  58  19  40  69  12  88  15  18   4  46  43  17  38
  31  62  32  86  26  41   7  16 100   6  61  45   1  65  96  11  78  42
   9  72  79  95  97  14  13  39  87  21  54  98  48  37  52  73  68  80
  93  47  29  60  36  94  25  57  33  84  55  92  27   8  10  91  89  74
  24  85  67  70  77  56  99  71  53  28  22  34  82  81  66  59  75  51
  90   5   3  30   2  49  63  83  64  35] 

[ 97  23  74   8  22  51  99  49  75  66  44  69  70  61   5  15  91  87
  92  33  27  31  62  54  73  60  53  63  93  36  12  11  71  81  47  65
  37  98  84   6  55  20  79  50   2  89  90  29  76   4   9  17  41  38
  30 100  77   3  64  14  24  78  96  43  59  13  86  82  56  21  85  95
  46  16  10  52  26  58  40  19  80   1  72  83  68  28  18  25  34   7
  94  45  67  35  57  39  88  48  42  32] 

[ 97  23  76  50  44  74

We also add a mutation function: 

In [55]:
def mutation(individual, mutation_rate = 0.2):
    # if random percentage is lower than the mutation rate, switch two random genes
    if random.uniform(0, 1) < mutation_rate:
        idx1 = random.randint(0, len(individual)-1)
        idx2 = random.randint(0, len(individual)-1)

        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]

    return individual

Finally, our executable GA class:

In [56]:
class Genetic_Algorithm():
    def __init__(self, students, num_individuals, groupsize):
        super(Genetic_Algorithm, self).__init__()

        self.students = students
        self.groupsize = groupsize
        self.num_individuals = num_individuals
        self.ids = students.ID.tolist()

    def run(self, episodes, replace='all', mutation_rate=0.2):

        self.epidodes = episodes
        self.population = Population(students, num_individuals, groupsize).create_initial_population()

        print("episode " + str(0) + ": mean fitness score: " + str(Fitness().mean_fitness(self.population)) + "; best individual fitness: " + str(Fitness().best_fitness(self.population)))

        if replace == 'all':
            for episode in range(episodes):
                new_pop = []
                # we get two new individuals by each step, so half the pop size
                for _ in range(len(self.population)//2):
                    # find two parents by tournament selection
                    p1, p2 = Crossover().tournament_selection(self.population,8)

                    # create two children by uniform order crossover
                    c1 = Crossover().uniform_order_crossover(p1,p2)
                    c2 = Crossover().uniform_order_crossover(p2,p1)
                    # do mutation
                    c1 = mutation(c1, mutation_rate)
                    c2 = mutation(c2, mutation_rate)
                    # add children to new population
                    new_pop.append(c1)
                    new_pop.append(c2)

                self.population = np.array(new_pop,dtype=int)

                print("episode " + str(episode+1) + ": mean fitness score: " + str(Fitness().mean_fitness(self.population)) + "; best individual fitness: " + str(Fitness().best_fitness(self.population)))

        elif list(replace):
            for episode in range(episodes):
                pop_indices_sorted_by_fitness = Fitness().indices_sorted_by_fitness(self.population)
                
                # we get two new individuals by each step, so half the pop size
                for _ in range(replace//2):
                    # find two parents by tournament selection
                    p1, p2 = Crossover().tournament_selection(self.population,8)

                    # create two children by uniform order crossover
                    c1 = Crossover().uniform_order_crossover(p1,p2)
                    c2 = Crossover().uniform_order_crossover(p2,p1)
                    # do mutation
                    c1 = mutation(c1)
                    c2 = mutation(c2)
                    # add children to new population
                    next_idx = pop_indices_sorted_by_fitness.pop(0)
                    self.population[next_idx] = c1
                    next_idx = pop_indices_sorted_by_fitness.pop(0)
                    self.population[next_idx] = c2

                print("episode " + str(episode+1) + ": mean fitness score: " + str(Fitness().mean_fitness(self.population)) + "; best individual fitness: " + str(Fitness().best_fitness(self.population)))

In [ ]:
Genetic_Algorithm(students, num_individuals, groupsize).run(episodes=10, replace='all', mutation_rate=0.2)

episode 0: mean fitness score: 16.183; best individual fitness: 19.592
episode 1: mean fitness score: 17.445; best individual fitness: 19.787
episode 2: mean fitness score: 17.581; best individual fitness: 20.687
episode 3: mean fitness score: 18.414; best individual fitness: 21.167
